In [2]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
tf.get_logger().setLevel(tf.logging.ERROR)

from src.models import LeNet
from src.data_utils import get_mnist

print(f"GPU available: {tf.test.is_gpu_available()}")

(X_train, y_train), (X_test, y_test) = get_mnist()

# prepare the model
lenet = LeNet(
    input_shape=X_train.shape[1:],
    num_classes=10,
)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
GPU available: True


# Train

In [3]:
%%time

epochs=20
mc_rate=0.5
lenet.set_mc_dropout_rate(mc_rate)
lenet.train(X_train, y_train, X_test, y_test, epochs=epochs, verbose=1)

# lenet.save_model(f'lenet-{mc_rate}-{epochs}')

Training with mc_dropout_rate = 0.5.

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 6s 106us/step - loss: 0.1524 - acc: 0.9527 - val_loss: 0.0646 - val_acc: 0.9803
Epoch 2/20
60000/60000 [==============================] - 5s 80us/step - loss: 0.0685 - acc: 0.9794 - val_loss: 0.0592 - val_acc: 0.9807
Epoch 3/20
60000/60000 [==============================] - 6s 100us/step - loss: 0.0524 - acc: 0.9843 - val_loss: 0.0592 - val_acc: 0.9844
Epoch 4/20
60000/60000 [==============================] - 6s 92us/step - loss: 0.0482 - acc: 0.9855 - val_loss: 0.0577 - val_acc: 0.9842
Epoch 5/20
60000/60000 [==============================] - 6s 101us/step - loss: 0.0405 - acc: 0.9872 - val_loss: 0.0735 - val_acc: 0.9812
Epoch 6/20
60000/60000 [==============================] - 6s 94us/step - loss: 0.0412 - acc: 0.9874 - val_loss: 0.0704 - val_acc: 0.9806
Epoch 7/20
60000/60000 [==============================] - 7s 120us/step - loss: 0.0319 

In [6]:
from gym_eli.envs import EliEnv
import gym

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2

In [5]:
mask = 1000
X_train = X_train[:mask]
y_train = y_train[:mask]

# The algorithms require a vectorized environment to run
env = DummyVecEnv([lambda: EliEnv(net=lenet,
                                  confidence_rate=0.5, 
                                  X_train=X_train, 
                                  y_train=y_train,
                                  )])

in init


# Train Agent

In [7]:
%%time
model = PPO2(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=500, tensorboard_log="ppo2_tensorboard")


# obs = env.reset()
# for i in range(2000):





Instructions for updating:
Use keras.layers.flatten instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


in reset


  3%|▎         | 20/687 [00:00<00:03, 196.58it/s]

in step
in take action
action=687


  6%|▌         | 32/522 [00:00<00:01, 314.99it/s]

in reset
in step
in take action
action=522


...

in reset
in step
in take action
action=716


  9%|▊         | 36/415 [00:00<00:01, 354.98it/s]

in reset
in step
in take action
action=415


100%|██████████| 415/415 [00:02<00:00, 200.31it/s]

in reset
-------------------------------------
| approxkl           | 0.0004475885 |
| clipfrac           | 0.0          |
| explained_variance | -0.000127    |
| fps                | 0            |
| n_updates          | 3            |
| policy_entropy     | 6.9058003    |
| policy_loss        | 0.0055909366 |
| serial_timesteps   | 384          |
| time_elapsed       | 624          |
| total_timesteps    | 384          |
| value_loss         | 491956.56    |
-------------------------------------
CPU times: user 16min 54s, sys: 1min 5s, total: 17min 59s
Wall time: 15min 49s
